# Teoria

### Czym są pipeline'y:
```
"The pipeline is a Python scikit-learn utility for orchestrating machine learning operations. 
Pipelines function by allowing a linear series of data transforms to be linked together, resulting in a measurable modeling process"
```
W uczeniu maszynowym bardzo rzadko zdarza się, by dane które posiadamy od razu nadawały się do przekazania do modelu ML.
Przed tym przeważnie niezbędne są różne przekształcenia jak np.:
- podmiana kolumn tekstowych na binarne/"gorącojedynkowe" 
- wypełnienie nulli w numerycznych biorąc mediany
- usunięcie części wierszy
- normalizacja
...

Wszystkie te kroki wchodzą w proces znany jako ETL (Extract-Transform-Load) i mogą być wykonywane wywołując poszczególne bloki kodu manualnie, jeden po drugim.

Pipeliny są mechanizmem który automatyzuje ten proces
- zapewniając pewną standaryzację implementacji 
- zajmując się przekazaniem danych z wyjścia jednego kroku na wejście kolejnego.

### Estymatory, Transformatory, Predyktory - pipeline'owy słowniczek
Estymatory, transformatory i predyktory to zbiór ustandaryzowanych funkcji stanowiących jakąś część etapu przygotowania danych/modelu i wykonania predycji

- __Transformatory (transformers)__ - najczęściej pierwszy etap przygotowania danych - transformatory przekształcają dane w różny sposób. Mogą zmieniać ich rozmiar, usuwać błędy lub ujednolicać wartości. Ich głównym celem jest przygotowanie danych wejściowych dla kolejnych etapów. Proces transformacji jest przeprowadzany za pomocą metody transform() a zostaje zwrócony przekształcony zbiór danych. Przykłady: StandardScaler(), PCA() <br>
NOTKA: Transformatory, są specjalnym typem estymatora i dlatego potrzebują zdefiniowania funkcji .fit() choć sama funkcja nie musi nic robić

- __Estymatory (estimators) - funkcje oszacowujące__ - Każdy obiekt zdolny do szacowania pewnych parametrów na podstawie zbioru danych jest zwany estymatorem. Może to być zarówno proste działanie, jak np. imputer wyliczający medianą z kolumny, ale i model klasyfikacyjny. Estymatory uczą się na danych treningowych i tworzą model, który będzie używany do przewidywania na danych testowych. Przykłady: inputer, DecisionTreeClassifier

- __Predyktory (predictors) - funkcje prognostyczne__ - szczególny rodzaj estymatora, który pozwala na przewidywanie jakichś wartości. Predyktory wymagają wytrenowania modelu na danych treningowych (metodą fit()) a później używają tak wytrenowanego modelu do przewidywania wyników (metodą predict()) Przykłady: LinearRegression, LogisticRegression, KNeighborsClassifier.

### Czy pipeline'y są zawsze potrzebne?
Nie - choć używanie ich wygląda profesjonalnie i dla niektórych może wejść w nawyk to czasami może stanowić narzędzie, którego implementacja (szczgólnie dla mniej doświadczonych z pipeline'ami użytkowników) jedynie wydłuży proces przygotowania i wdrożenia modelu.

#### Kiedy nie warto?
- __Małe i proste zadania__ : W przypadku bardzo prostych i małych zadań, które składają się tylko z jednego lub dwóch etapów przetwarzania danych, używanie potoków może być zbędne i zwiększać złożoność kodu
- __Jednorazowe użycie modelu__ - przy implementacji modelu, który będzie miał jednorazowo spełnić jakieś zadanie ale nie będzie wielokrotnie używany może być szybsze uzyskanie wyniku bez dodatkowej implementacji pipeline'a
- __Potrzeba podglądania danych na różnych etapach__ - kiedy chcemy mieć łatwą możliwość podejrzenia stanu datafrema'u po każdym kroku (w szczególności gdy jesteśmy poczatkujący i nie jesteśmy całkowicie pewni jak wyglądają nasze dane po każdej transformacji)
- __Potrzeba bardziej elastycznego podejścia__ - W przypadku, gdy potrzebujesz bardziej elastycznego podejścia do przetwarzania danych, które pozwoli na łatwe dodawanie i usuwanie etapów przetwarzania danych w zależności od potrzeb, bardziej odpowiednim podejściem może być ręczne wywoływanie metod przetwarzania danych w kodzie niż implementowanie obsługi parametrów dla poszczególnych estymatorów.

#### Kiedy warto?
- __Kiedy będziemy pracować ze strumieniowanymi danymi__ - praca z danymi w formie "live" w naturalny sposób będzie wymagała częstego uruchamiania każdego kroku potoku wielokrotnie. Przekazanie danych do potoku usprawnia ten proces i czyni go czytelniejszym
- __Kiedy za różne elementy potoku odpowiadają różni ludzie__ - w większych zespołach może zdarzyć się tak, że będziemy odpowiedzialni tylko zaczęść obróbki danych - wtedy użycie potoku poniekąd wymusza utrzymanie spójnego formatu danych co może ułatwić współpracę
- __Dużo eksperymentów z hiperparametrami__ - w zależności od sposobu implementacji bez-potoków, używanie ich może okzać się wygodniejsze, jeżeli chcemy przetestowac wiele wariantów modelu bo możliwym będzie skinfigurowanie parametrów potoku w jednym miejscu
- __Wymuszenie organizacji kodu__ - choć poszczególne etapy przygotowania danych można samodzielnie obudować w funkcje/metody, potoki w pewnym sensie wymuszają to na nas co może stanowić dobry nawyk, w szczególności dla ludzi nie potrafiących narzucić sobie samodzielnie odpowiedniej dbałości
- __Potrzeba większej wydajności★__ - (★nie testowałem) podobno przy odpowiedniem implementacji możliwe jest równoległe przetwarzanie części danych przez różne etapy ujęte w potoku co przyśpiesza całkowity czas przetworzenia danych; ponadto - podobno część transformatorów może mieć lepszą implementację fit_and_transform niż wywoływanie tego oddzielnie. 
- __Kiedy chcemy, żeby nasz kod wyglądał bardziej profesjonalnie :)__ - pipeline'y zostały stworzone specjalnie do pracy z obróbką danych i ich znajomość oraz umiejętność zastosowania dobrze świadczy o naszym zorientowaniu w branży

### OOP (Programowanie obiektowe) vs Pipelines
Jako, że w pythonie "wszystko jest obiektem" - tak samo jest z pipeline'ami.<br>
W zależności od tego, jak zaimplementujemy nasze metody / klasy - możemy osiągnąć podobny (lub identyczny) efekt jaki zapewniają pipeline'y.

Skorzystanie z pakietu sklearn i pipeline'ów pozwala na dopasowanie się do pewnych standardów, może zwiększyć wydajność ale jednocześnie odbiera odrobinę swobody.

### sklearn.pipeline.Pipeline vs sklearn.pipeline.make_pipeline 
__Pipeline__ wymaga definiowania przez nas nazw poszczególnych kroków przy definiowaniu potoku:
```
pipe = Pipeline([('std_sc', StandardScaler()),
                 ('svc_', SVC(gamma='auto'))])
```
Nadane nazwy kroków: `[std_sc, svc_]`<br><br>

__make_pipeline__ to konstruktor Pipeline'ów, który sam nadaje nazwy (biorąc nazwę użytego estyamtora jako lowercase):
```
mp = make_pipeline(StandardScaler(),
                   SVC(gamma='auto')) 
```
Otrzymane nazwy kroków: `[standardscaler, svc]`

# Przykład użycia w kodzie
Źródło: https://youtu.be/h1BnRBzYjYY
- Z video wzięto przykładowe dane oraz pierwotną wersję wdrożonych encoderów
- Część encoderów zmieniłem/przebudowałem wersjami własnymi/alternatywnymi dla lepszej czytelności kodu
- Dodano znacznie więcej przykładów implementacji (jak użycie make_pipeline czy column_transformer)

In [1]:
import pandas as pd
data ={
    "Name": ["Anna", "Bob", "Charlie", "Diana", "Eric", ],
    "Age": [20, 34, 23, None, 33, ],
    "Gender": ["f", "m", "m", "f", "m", ],
    "Job": ["Programmer", "Writter", "Cook", "Programmer", "Teacher", ]
}
df = pd.DataFrame(data)
df

,Name,Age,Gender,Job
0,Anna,20.0,f,Programmer
1,Bob,34.0,m,Writter
2,Charlie,23.0,m,Cook
3,Diana,NaN,f,Programmer
4,Eric,33.0,m,Teacher


#### Poniżej przykład obrazujący przetwarzanie danych bez/z wykorzystaniem pipeline'ów
- Przykład ma przede wszystkim __zademonstrować działanie pipelinów__ (ręczne wywoływanie estymatorów vs użycie pipeline'ów)
- Przykład ma przybliżyć dwie metody tworzenia (Pipeline i make_pipeline)
- Przykład ma obrazować jak bardzo użycie pipeline'u może zwiększyć objętość kodu
    ```
    UWAGA: Zademonstrowany przykład używa estymatorów w stosunku do konkretnych kolumn.
    • Przeważnie używa się tych samych estymatorów dla wszystkich kolumn tego samego typu (dla numerycznych jednych, dla kategorycznych drugich)
    • W przypadku nie-wskazywania konkretnych nazw kolumn definicje definicje estymatorów byłyby krótsze
    • Pracując na danych w formie Arrayki (bez zachowania struktury kolumn dataframe'u) można używać więcej gotowych estymatorów co pozwala pominąć część definicji

![](../img/2023-04-05-20-33-10.png)

In [2]:
from sklearn.impute import SimpleImputer

df = pd.DataFrame(data)

# Drop name feature
df = df.drop(['Name'], axis=1)

# Impute ages
imputer = SimpleImputer(strategy='mean')
df['Age'] = imputer.fit_transform(df[['Age']])

# Convert gender to numeric
df['Gender'] = df['Gender'].map({'m': 0, 'f':1})

# OneHotEncode Jobs
df = pd.get_dummies(df, columns = ['Job'], dtype=int)
df

,Age,Gender,Job_Cook,Job_Programmer,Job_Teacher,Job_Writter
0,20.0,1,0,1,0,0
1,34.0,0,0,0,0,1
2,23.0,0,1,0,0,0
3,27.5,1,0,1,0,0
4,33.0,0,0,0,1,0


### TransformerMixin sprawia, że nie musimy definiować metody fit_transform a jest ona dziedziczona z tej klasy

In [3]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer

class NameDropper(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self # zdefiniowanie fit'a jest koniecznie, ale sam fit nie musi nic robić

    def transform(self, X):
        return X.drop(['Name'], axis=1)

class AgeImputer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        imputer = SimpleImputer(strategy="mean") 
        #Imputer could be used by itself in pipeline, 
        # but it requires only numeric columns while our dataframe hase also categorical ones
        X['Age'] = imputer.fit_transform(X[['Age']])
        return X

class GenderEncoder(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X['Gender'] = X['Gender'].map({'m': 0, 'f':1})
        return X

class OneHotEncoderForGivenColumns(BaseEstimator, TransformerMixin):  
    def __init__(self, columns_to_encode=None):
        if columns_to_encode is None:
            self.columns_to_encode = []
        else:
            self.columns_to_encode = columns_to_encode
        self.encoded_columns = None

    def fit(self, X, y=None):
        self.encoded_columns = pd.get_dummies(X, columns=self.columns_to_encode).columns
        return self

    def transform(self, X):
        X_new = pd.get_dummies(X, columns=self.columns_to_encode, dtype=int)
        return X_new.reindex(columns=self.encoded_columns, fill_value=0)

In [4]:
df2 = pd.DataFrame(data)

dropper = NameDropper()
imp = AgeImputer()
genc = GenderEncoder()
ohe = OneHotEncoderForGivenColumns(['Job'])

df2 = ohe.fit_transform(genc.fit_transform(imp.fit_transform(dropper.fit_transform(df2))))
df2

,Age,Gender,Job_Cook,Job_Programmer,Job_Teacher,Job_Writter
0,20.0,1,0,1,0,0
1,34.0,0,0,0,0,1
2,23.0,0,1,0,0,0
3,27.5,1,0,1,0,0
4,33.0,0,0,0,1,0


In [5]:
df3 = pd.DataFrame(data)

from sklearn.pipeline import Pipeline
pipe = Pipeline([
    ('dropper', NameDropper()),
    ('imputer', AgeImputer()),
    ('gender_encoder', GenderEncoder()),
    ('one_hot_encoder', OneHotEncoderForGivenColumns(['Job'])),
])
df3 = pipe.fit_transform(df3)
df3

,Age,Gender,Job_Cook,Job_Programmer,Job_Teacher,Job_Writter
0,20.0,1,0,1,0,0
1,34.0,0,0,0,0,1
2,23.0,0,1,0,0,0
3,27.5,1,0,1,0,0
4,33.0,0,0,0,1,0


In [6]:
df4 = pd.DataFrame(data)

from sklearn.pipeline import make_pipeline

pipe = make_pipeline(
    NameDropper(),
    AgeImputer(),
    GenderEncoder(),
    OneHotEncoderForGivenColumns(['Job']),
)
df4 = pipe.fit_transform(df4)
df4

,Age,Gender,Job_Cook,Job_Programmer,Job_Teacher,Job_Writter
0,20.0,1,0,1,0,0
1,34.0,0,0,0,0,1
2,23.0,0,1,0,0,0
3,27.5,1,0,1,0,0
4,33.0,0,0,0,1,0


# Inne sposoby przydziału encoderów:
- Głównie, żeby zobrazować, że przy odpowiedniem implementacji objętość kodu wcale nie musi być tak dużą
- Tracimy jednak możliwość podglądu w postaci dataframe'u poszczególnych nazw kolumn
```
UWAGA: Z racji, że i tak nie zwracamy nazw kolumn, część własnych implementacji (jak OneHotEncoder) zastąpiono wersją z pakietu 
   (która nie miała tej funkcjonalności więc wczesniej nie mogliśmy jej użyć chcąc mieć wynik w postaci dataframe'u z nazwami)
   ```

![](../img/2023-04-05-21-34-52.png)

# Wariant z rozdzieleniem typów kolumn na oddzielne pipeline'y

In [7]:
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.attribute_names]

from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

In [8]:
from sklearn.pipeline import FeatureUnion
df5 = pd.DataFrame(data)

num_pipeline = Pipeline([
    ("select_numeric", DataFrameSelector(["Age"])),
    ("imputer", SimpleImputer(strategy="median")),
])

mapped_pipeline = Pipeline([
    ("select_map", DataFrameSelector(["Gender"])),
    ('map', OrdinalEncoder(categories=[['m', 'f']])),
])

cat_pipeline = Pipeline([
    ("select_cat", DataFrameSelector(["Job"])),
    ("cat_encoder", OneHotEncoder(sparse_output=False)),
])

preprocess_pipeline = FeatureUnion(transformer_list=[
    ("num_pipeline", num_pipeline),
    ("mapped_pipeline", mapped_pipeline),
    ("cat_pipeline", cat_pipeline),
])

preprocess_pipeline.fit_transform(df5)


array([[20.,  1.,  0.,  1.,  0.,  0.],
       [34.,  0.,  0.,  0.,  0.,  1.],
       [23.,  0.,  1.,  0.,  0.,  0.],
       [28.,  1.,  0.,  1.,  0.,  0.],
       [33.,  0.,  0.,  0.,  1.,  0.]])

# Wariant z użyciem ColumnTransformera 

In [9]:
# Ten wariant nie wymaga żadnego definiowania własnych encoderów, ale traci informacje o kolumnach po przetworzeniu
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

In [10]:
df6 = pd.DataFrame(data)

column_trans = ColumnTransformer(
    [
        ('imputer', SimpleImputer(strategy='median'), ['Age']),
        ('map', OrdinalEncoder(categories=[['m', 'f']]), ['Gender']),
        ('ohe', OneHotEncoder(sparse_output=False), ['Job']),
    ],
    remainder='drop')

column_trans.fit_transform(df6)

array([[20.,  1.,  0.,  1.,  0.,  0.],
       [34.,  0.,  0.,  0.,  0.,  1.],
       [23.,  0.,  1.,  0.,  0.,  0.],
       [28.,  1.,  0.,  1.,  0.,  0.],
       [33.,  0.,  0.,  0.,  1.,  0.]])